# Face Detection w/ web stream| photo| video

[link](https://www.pyimagesearch.com/2018/02/26/face-detection-with-opencv-and-deep-learning/)

After opencv 3.3, Ryabnikov includes accurate, deep learning based face detector [face_detector](https://github.com/opencv/opencv/tree/master/samples/dnn/face_detector)

need two file: 

- .prototxt: defines the model architecture

- .caffemodel: contains the weights for the actual layers

### Face Detection for Photo

In [1]:
import numpy as np
import argparse
import cv2

args = {'prototxt': './simple-object-tracking/deploy.prototxt', 
    'model': './simple-object-tracking/res10_300x300_ssd_iter_140000.caffemodel', 
    'image':'./test.jpeg', 
    'confidence': .5}

net = cv2.dnn.readNetFromCaffe(args['prototxt'], args['model'])

# load the input image and construct an input blob for the image by resizing to a fixed 300 * 300 pixels and then normalizing it
image = cv2.imread(args['image'])
(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

# pass the blob through the network and obtain the detections and predictions
net.setInput(blob)
detections = net.forward()

# loop over the detections 
for i in range(0, detections.shape[2]): 

    # extract the confidence associated with the prediction
    confidence = detections[0, 0, i, 2]

    # filter out weak detections by ensuring the 'confidence' is greater than the minimum confidence
    if confidence > args['confidence']: 

        # compute the (x, y) - coordinates of the bounding box for the object
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype('int')

        # draw the bounding box of the face along with the associated probability
        text = '{:.2f}%'.format(confidence * 100)    
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
        cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, .45, (0, 0, 255), 2)

cv2.imshow('Output', image)
cv2.waitKey(0)

32

### Face detection for video

In [2]:
from imutils.video import FPS
import numpy as np
import imutils
import cv2

args = {'prototxt': './simple-object-tracking/deploy.prototxt', 
    'model': './simple-object-tracking/res10_300x300_ssd_iter_140000.caffemodel', 
    'video':'./test.mp4', 
    'confidence': .5}


net = cv2.dnn.readNetFromCaffe(args['prototxt'], args['model'])

# open a pointer to the video stream and start the FPS timer
stream = cv2.VideoCapture(args['video'])
fps = FPS().start()

# loop over frames from the video file stream
while True:

    # grab the frame from the threaded video file stream
    (grabbed, frame) = stream.read()

    # if the frame was not grabbed, then we have reached the end of the stream
    if not grabbed: 
        break

    # resize the frame and covert it to grayscale (while still retaining 3 channels)
    frame = imutils.resize(frame, width = 400)

    # grab the frame dimensions and covert it to a blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

    # pass the blob throught the network and obtain the detections and preditions
    net.setInput(blob)
    detections = net.forward()

    #loop over the detections
    for i in range(0, detections.shape[2]): 

        # extract the confidence associated with the prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the 'confidence' is greater than the minimum confidence
        if confidence < args['confidence']: 
            continue
        
        # compute the (x, y)- coordinates of the bounding obx for the object
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype('int')

        # draw the bounding box of the face along with the associated probability
        text = '{:.2f}%'.format(confidence * 100)    
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)
        cv2.putText(frame, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, .45, (0, 0, 255), 2)

    # show the frame and update the FPS counter
    cv2.imshow('Frame', frame)
    key = cv2.waitKey(1) & 0xFF
    fps.update()

    # if the 'q' key was pressed, break from the loop
    if key == ord('q'):
        break

# stop the timer and display FPS information
fps.stop()
stream.release()
cv2.destroyAllWindows()


# Deal with Images


[link](https://www.pyimagesearch.com/2018/07/19/opencv-tutorial-a-guide-to-learn-opencv/)

- deal cv2 and imutils with basic image skills

- deal cv2 with the erode and mask etc

# Edge Detection

[link](https://www.pyimagesearch.com/2019/03/04/holistically-nested-edge-detection-with-opencv-and-deep-learning/)


### Holistically-Nested Edge Detection

In [1]:
class CropLayer(object): 
    def __init__(self, params, blobs): 
        # initialize our starting and ending (x, y) - coordinates of the crop
        self.startX = 0
        self.startY = 0
        self.endX = 0
        self.endY = 0
    
    def getMemoryShapes(self, inputs): 
        # the crop layer will receive two inputs -- we need to crop the first input blob to match the shape of the second one, keeping the batch size and number of channels
        (inputShape, targetShape) = (inputs[0], inputs[1])
        (batchSize, numChannels) = (inputShape[0], inputShape[1])
        (H, W) = (targetShape[2], targetShape[3])

        # compute the starting and ending crop coordinates
        self.startX = int((inputShape[3] - targetShape[3]) / 2)
        self.startY = int((inputShape[2] - targetShape[2]) / 2)
        self.endX = self.startX + W
        self.endY = self.startY + H

        # return the shape of the volume (we will preform the actual crop during the forward pass)
        return [[batchSize, numChannels, H, W]]
    
    def forward(self, inputs): 
        # use the derived (x, y)-coordinates to perform the crop
        return [inputs[0][:, :, self.startY:self.endY, self.startY: self.endX]]

In [3]:
import cv2
import os

args = {'image': 'test_1.jpeg', 'edge_detector': './holistically-nested-edge-detection/hed_model'}

roots = './holistically-nested-edge-detection/hed_model'
protoPath = os.path.join(args['edge_detector'], 'deploy.prototxt')
modelPath = os.path.join(args['edge_detector'], 'hed_pretrained_bsds.caffemodel')

net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# register our new layer with the model
cv2.dnn_registerLayer('Crop', CropLayer)

# load the input image and grab its dimensions
image = cv2.imread(args['image'])
(H, W) = image.shape[:2]

# convert the image to grayscale, blur it, and perform Canny edge detection
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
canny = cv2.Canny(blurred, 30, 150)

# construct a blob out of the input image for the Holistically-Nested Edge Detector
blob = cv2.dnn.blobFromImage(image, scalefactor = 1.0, size = (W, H), mean = (104.00698793, 116.66876762, 122.67891434), swapRB = False, crop = False)

# set the blob as the input to the network and perform a forward pass to compute the edges
net.setInput(blob)
hed = net.forward()
hed = cv2.resize(hed[0, 0], [W, H])
hed = (255 * hed).astype('uint8')

# show the output edge detection results for Canny and Holistically-Nested Edge Detection
cv2.imshow('Input', image)
cv2.imshow('Canny', canny)
cv2.imshow('HED', hed)
cv2.waitKey(0)